# PAN16

In [1]:
import os
from argparse import ArgumentParser
from sklearn.externals import joblib
from tictacs import from_recipe
from pan import ProfilingDataset
import dill
import cPickle as pickle
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix


infolder = "../DATA/pan16-author-profiling-training-dataset-2016-04-25/pan16-author-profiling-training-dataset-english-2016-02-29/"
outfolder = "models/"
print('Loading dataset->Grouping User texts.\n')
dataset = ProfilingDataset(infolder)
print('Loaded {} users...\n'.format(len(dataset.entries)))
# get config
config = dataset.config
tasks = config.tasks
print('\n--------------- Thy time of Running ---------------')
all_models = {}
for task in tasks:
    print('Learning to judge %s..' % task)
    # load data
    X, y = dataset.get_data(task)

Loading dataset->Grouping User texts.

Loaded 436 users...


--------------- Thy time of Running ---------------
Learning to judge age..
Learning to judge gender..


In [ ]:
# Instance Based
from pan import ProfilingDataset, createDocProfiles, create_target_prof_trainset
from pan import preprocess

task = 'gender'
docs = createDocProfiles(dataset)
X, y = create_target_prof_trainset(docs, task)
print len(X)
#print X[0]
X = preprocess.preprocess(X)

In [5]:
# Profile Based
#reload(preprocess)
#reload(features)
from pan import features
from pan import preprocess
X, y = dataset.get_data('age')
#X, y = dataset.get_data('gender')
print len(X)
#print X[0]
#X = preprocess.preprocess(X)
#print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
#print X[0]

436


## AGE

In [3]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from pan.features import SOA_Model2
from pan import features
grams3 = TfidfVectorizer(analyzer='word', ngram_range=[1,1], max_features=7500, stop_words='english')
# soa = features.SOA_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None)
combined = FeatureUnion([('3grams', grams3)])
#svm = LinearSVC(C=0.001, dual=False, class_weight='balanced')
svm = SVC(kernel='linear', C=1, gamma=1, class_weight='balanced', probability=False)
#pipe = Pipeline([('3grams',grams3), ('svm', svm)])
pipe = Pipeline([('combined',combined), ('svm', svm)])
pipe.steps

[('combined', FeatureUnion(n_jobs=1,
         transformer_list=[('3grams', TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_features=7500, min_df=1,
          ngram_range=[1, 1], norm=u'l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))],
         transformer_weights=None)),
 ('svm', SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

## SOAC

In [25]:
soac = features.SOAC_Model2(max_df=1.0, min_df=1, tokenizer_var=3, max_features=None)
svm = SVC(kernel='rbf', C=0.1, gamma=1, class_weight='balanced', probability=False)
pipe1 = Pipeline([('soac',soac), ('svm', svm)])
pipe1.steps

[('soac', SOAC_Model2(max_df=1.0, max_features=None, min_df=1, thres=0.1,
        tokenizer_var=3)),
 ('svm', SVC(C=0.1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

In [26]:
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
num_folds = 4
split = 0.25
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y)
for i, x in enumerate(X_train):
    if len(x)==0:
        X_train.remove(x)
        y_train.remove(y_train[i])
for i, x in enumerate(X_cv):
    if len(x)==0:
        X_cv.remove(x)
        y_cv.remove(y_cv[i])
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)
params = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100]}
params = {}
grid_cv = GridSearchCV(pipe1, param_grid=params, verbose=1, n_jobs=-1, cv=num_folds, refit=True)
grid_cv.fit(X_train, y_train)
#grid_cv.fit(X_train, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
pipe1 = grid_cv.best_estimator_
#pipe.fit(X_train, y_train)


326 110 436 436
Fitting 4 folds for each of 1 candidates, totalling 4 fits
0.438650306748
{}


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   20.0s finished


In [27]:
import copy
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(grid_cv.best_score_)
print(grid_cv.best_params_)
predict = grid_cv.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

0.438650306748
{}
Accuracy : 0.454545454545
Confusion matrix :
 [[ 0  2  5  0  0]
 [ 0  8 27  0  0]
 [ 0  6 40  0  0]
 [ 0  0 18  2  0]
 [ 0  0  2  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         7
      25-34       0.50      0.23      0.31        35
      35-49       0.43      0.87      0.58        46
      50-64       1.00      0.10      0.18        20
      65-xx       0.00      0.00      0.00         2

avg / total       0.52      0.45      0.38       110



In [28]:
#a = pipe1.steps[0][1]
a = pipe1.steps[0][1]
#a = grid_search.best_estimator_.steps[0][1]
feature_names = []
soa_feat_names = ["soa_prob_"+str(i) for i in range(0, len(set(y)))]
soac_feat_names = ["soac_prob_"+str(i) for i in range(0, len(set(y)))]
#feature_names += soa_feat_names
feature_names += soac_feat_names
print len(feature_names)
feature_names = [feat.encode('utf-8') for feat in feature_names]
import pandas as pd
data = pd.DataFrame(a.transform(X), columns=feature_names)
data["class"] = y
data['class_pred'] = pipe1.predict(X)
data['text'] = X
print(data.describe())


data_cv = pd.DataFrame(a.transform(X_cv), columns=feature_names)
data_cv["class"] = y_cv
data_cv['class_pred'] = pipe1.predict(X_cv)
data_cv['text'] = X_cv
print(data_cv.describe())

data_train = pd.DataFrame(a.transform(X_train), columns=feature_names)
data_train["class"] = y_train
data_train['class_pred'] = pipe1.predict(X_train)
data_train['text'] = X_train
print(data_train.describe())

5
       soac_prob_0  soac_prob_1  soac_prob_2  soac_prob_3  soac_prob_4
count   436.000000   436.000000   436.000000   436.000000   436.000000
mean      0.535937     0.534637     0.514828     0.538711     0.533375
std       0.379431     0.546550     0.622630     0.464879     0.348995
min       0.000000     0.000000     0.000000     0.000000     0.000000
25%       0.096137     0.000000     0.000000     0.042144     0.174264
50%       0.576461     0.446715     0.061299     0.585766     0.566729
75%       0.802871     0.994705     0.998661     0.886472     0.775899
max       1.904933     2.669076     2.525765     2.162857     1.801661
       soac_prob_0  soac_prob_1  soac_prob_2  soac_prob_3  soac_prob_4
count   110.000000   110.000000   110.000000   110.000000   110.000000
mean      0.067718     0.071014     0.053022     0.072085     0.068183
std       0.058208     0.076580     0.092235     0.081902     0.056019
min       0.000000     0.000000     0.000000     0.000000     0.000000
25% 

In [33]:
#soacc = a.transformer_list[0][1]
voc = a.counter.vocabulary_
print 'Voc: ' + str(len(voc))
print a.term_table.shape
#terms= ['marriage', 'pension']
#graph_matrix = numpy.zeros([len(terms), soacc.term_table.shape[1]])
j = 1
wanted_class = 0
for term, index in voc.iteritems():
    l = list(a.term_table[index,:])
    if l.index(min(l))==wanted_class and  min(l)<0.02 and min(l)!=0:
        print term
        print l
        j += 1
    if j==1000:
        break

Voc: 343008
(343008, 5)
lollipop
[0.019742405110670286, 0.25448890828494231, 0.31414234673065339, 0.22350466780311767, 0.18812167207061642]
brunswick
[0.017357372647224204, 0.28287618158478967, 0.28741583499728607, 0.22389793205893641, 0.18845267871176358]
swiffer
[0.0040816136286238817, 0.27480276597456338, 0.31212830952441961, 0.22207172913245718, 0.18691558173993592]
ukjobs
[0.0063890298121699581, 0.28112003909538069, 0.3027002616503236, 0.22250793637398109, 0.18728273306814472]
hireme
[0.0085738328835328215, 0.28146984813785053, 0.29965573042097021, 0.2227848120048162, 0.18751577655283019]
informa
[0.016604240634903421, 0.28275559754152441, 0.28846532726566987, 0.22380248917726592, 0.18837234538063646]
arians
[0.0096863781747230579, 0.28164797787783341, 0.29810539440785494, 0.22292580259722636, 0.18763444694236223]
harto
[0.0051805983398636587, 0.28092655700916486, 0.30438421541448357, 0.2223547943216829, 0.18715383491480483]
winnie
[0.019834836134910081, 0.28075361721848041, 0.312

In [43]:
import operator
#a = pipe1.steps[0][1]
#a = trained_models[1].steps[0][1]
a = grid_cv.best_estimator_.steps[0][1]
#print a
n_gram = a.transformer_list[0][1]
dict1 = n_gram.vocabulary_
list1 = zip(dict1.values(), dict1.keys())
list1.sort(key=operator.itemgetter(0))
feature_names = [word[1] for word in list1]

In [46]:
#vectorized = a.transform(X).toarray()
labels = sorted(list(set(y)))
for lab in labels

In [47]:
import pandas as pd
data = pd.DataFrame(vectorized)
data["class"] = y
data['class_pred'] = grid_cv.predict(X)
data['text'] = X
print(data.describe())

             0           1           2           3           4           5     \
count  436.000000  436.000000  436.000000  436.000000  436.000000  436.000000   
mean     0.000354    0.000536    0.000086    0.000165    0.000097    0.002080   
std      0.001079    0.001140    0.001097    0.001948    0.000889    0.005572   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
75%      0.000138    0.000610    0.000000    0.000000    0.000000    0.001792   
max      0.014153    0.010610    0.021827    0.029561    0.016177    0.077100   

             6           7           8           9        ...            7490  \
count  436.000000  436.000000  436.000000  436.000000     ...      436.000000   
mean     0.000181    0.001221    0.001541    0.001463     ...        0.000102   
std      0.001079    0.0027

In [53]:
grouped = data.groupby('class')
means = grouped.mean().T

In [74]:
means['female']+means['male']

0       0.000708
1       0.001072
2       0.000172
3       0.000330
4       0.000194
5       0.004159
6       0.000362
7       0.002441
8       0.003081
9       0.002926
10      0.003445
11      0.003706
12      0.003779
13      0.003583
14      0.004068
15      0.006344
16      0.001381
17      0.000542
18      0.000598
19      0.000252
20      0.000340
21      0.003310
22      0.000375
23      0.000393
24      0.000187
25      0.000306
26      0.002589
27      0.000180
28      0.004944
29      0.000204
          ...   
7470    0.000351
7471    0.000260
7472    0.001973
7473    0.000537
7474    0.000317
7475    0.000503
7476    0.000251
7477    0.000233
7478    0.000265
7479    0.000262
7480    0.000254
7481    0.000396
7482    0.001921
7483    0.000246
7484    0.006391
7485    0.001169
7486    0.000179
7487    0.000275
7488    0.000273
7489    0.000443
7490    0.000203
7491    0.000333
7492    0.000087
7493    0.000111
7494    0.000822
7495    0.000364
7496    0.000288
7497    0.0003

In [72]:
gg = means['female']
#gg.sort(ascending=[0])
means2 = means - means['female']
result = means2.sort(['male'], ascending=[0])
result

,0,1,2,3,4,5,6,7,8,9,...,7492,7493,7494,7495,7496,7497,7498,7499,female,male
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
